In [1]:
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Sequential
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

import numpy as np

import pandas as pd


In [2]:
# carregar modelo word2vec
word2vec = KeyedVectors.load_word2vec_format('cbow_s100.txt')
vocab_size, embedding_size = word2vec.vectors.shape

In [3]:
# carregar dados de treinamento do CSV
df = pd.read_csv('train.csv', header=None, names=['texto', 'classe'])
X = df['texto'].values
y = df['classe'].values

In [5]:
# tokenizar o texto
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# padronizar o tamanho das sequencias
max_length = max([len(text) for text in X])
X = pad_sequences(X, maxlen=max_length, padding='post')

In [6]:
# construir modelo
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, weights=[word2vec.vectors], input_length=max_length, trainable=False))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(7, activation='softmax'))

In [7]:
# compilar modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# separar dados em treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, 7)
y_val = to_categorical(y_val, 7)

In [9]:
# treinar modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
26/26 [==============================] - 3s 35ms/step - loss: 1.6105 - accuracy: 0.4138 - val_loss: 0.9963 - val_accuracy: 0.6749
Epoch 2/10
26/26 [==============================] - 0s 11ms/step - loss: 0.5835 - accuracy: 0.8140 - val_loss: 0.3454 - val_accuracy: 0.9113
Epoch 3/10
26/26 [==============================] - 0s 11ms/step - loss: 0.1741 - accuracy: 0.9569 - val_loss: 0.2234 - val_accuracy: 0.9458
Epoch 4/10
26/26 [==============================] - 0s 12ms/step - loss: 0.0901 - accuracy: 0.9815 - val_loss: 0.2496 - val_accuracy: 0.9360
Epoch 5/10
26/26 [==============================] - 0s 15ms/step - loss: 0.0692 - accuracy: 0.9815 - val_loss: 0.1862 - val_accuracy: 0.9507
Epoch 6/10
26/26 [==============================] - 0s 12ms/step - loss: 0.0768 - accuracy: 0.9766 - val_loss: 0.1556 - val_accuracy: 0.9704
Epoch 7/10
26/26 [==============================] - 0s 13ms/step - loss: 0.0677 - accuracy: 0.9852 - val_loss: 0.2154 - val_accuracy: 0.9458
Epoch 8/10
26